In [5]:
import pandas as pd

In [6]:
import re

In [7]:
import numpy as np
import codecs

In [8]:
data = pd.DataFrame(pd.read_excel('microwave.xlsx'))

In [9]:
data = list(data['review_body'].values)

In [10]:
data = [re.sub(',|\.|\?|\(|\)|\d|\*|!|@|#|$|%|\^|&|<|>|:|;|\'|-','',str(i)) for i in data]
data = [s.lower() for s in data]
f = open('stopwords.txt')
with codecs.open('stopwords.txt','r',encoding="utf-8") as f:
    delList=f.read().split()
dictIndex={}
dictCount={}
for lines in data:
    for words in lines.split():
        if words in delList or len(words)<2:
            continue
        dictIndex[words] = dictIndex.get(words, len(dictIndex))
        dictCount[words] = dictCount.get(words,1) + 1
        
wordCount = pd.DataFrame(columns = ['index','word','count'])
wordCount['index'] = list(range(len(dictIndex)))
wordCount['word'] = list(dictCount.keys())
wordCount['count'] = list(dictCount.values())
poplist = list(wordCount[wordCount['count']<20].index)
wordCount = wordCount.drop(poplist)
wordCount.to_csv('ciyun.csv',index = False)


In [11]:
from gensim import corpora
from collections import defaultdict
texts = [[word for word in lines.lower().split() if word not in delList]for lines in data]
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [15]:
texts = [[token for token in text if frequency[token]>10] for text in texts]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
from gensim import models

In [14]:
tfidf = models.TfidfModel(corpus)

In [19]:
string = 'microwave match button'
string_bow = dictionary.doc2bow(string.lower().split())
string_tfidf = tfidf[string_bow]
model = models.Word2Vec(texts, size=100, window=5, min_count=1, workers=4)
print(model.wv['button'])
model.wv.similarity('microwave','button')

[ 0.10048281 -0.4666785  -0.4213975   0.11408072 -0.26269522 -0.37425998
 -0.47669774 -0.12157673  0.35743108  0.35257384  0.03412536 -0.29881
  0.02167001 -0.10214388 -0.17482023 -0.42803693  0.04219373 -0.40832585
  0.16006605  0.28736034 -0.14385273 -0.4110848  -0.4794865   0.02619536
  0.49194333 -0.55000085 -0.47597656  0.01875384 -0.03781598  0.54656345
  0.25239167 -0.04646043 -0.34408757 -0.3524025  -0.27879536 -0.11825086
 -0.1936868   0.2945022  -0.09906912  0.10426231 -0.26747677  0.7542405
  0.08951312  0.05132932  0.03019384  0.5548918   0.14467157  0.09636927
  0.05037932  0.01028826 -0.38490063  0.00668541 -0.36107025  0.06071957
  0.53087425 -0.34287232 -0.16185857 -0.03350376  0.22582659 -0.15810198
 -0.02291001  0.29635558 -0.08451406 -0.46215564  0.41914472  0.30808055
 -0.05002823 -0.42996377 -0.30900574 -0.05913643  0.21379635  0.10466556
  0.17034474 -0.02328461  0.4222541   0.03905042 -0.14677784  0.22300178
  0.23248169 -0.40000284  0.53697264 -0.45209444 -0.215

0.9996156